## Video Background Swapper

### Objetivo

Utilizando as mesmas técnicas utilizadas na troca de _background_ de imagens, deve-se criar um algoritmo que possibilite a detecção e troca de _backgrounds_ de vídeos que utilizam _chroma key_, ou seja, que possuem _background_ verde. Portanto, o algoritmo final utiliza a função `swap_image_background` do [algoritmo de troca de _backgrounds_ de imagens](./image_background_swapper.py).

### Explicação Geral

O algoritmo para troca de _background_ de vídeos foi desenvolvido utilizando as seguintes bibliotecas:

- A `cv2` (OpenCV) foi utilizado para ler o vídeo de entrada, processar cada frame (substituindo o _background_) e escrever o vídeo de saída (sem áudio);
- A `os` foi utilizado para manipular o vídeo de saída durante a etapa de inserção de áudio, tornando o vídeo de saída apenas em um arquivo temporário e logo após removendo-o, após o novo vídeo de saída estar pronto;
- A `sys` foi utilizada para ler os parâmetros inseridos na linha de comando, sendo eles o nome do vídeo de entrada e a imagem que contém o novo _background_;
- A função `swap_image_background` foi utilizada para trocar o _background_ verde de cada frame do vídeo, conforme ele foi lido;
- A `moviepy.editor` foi utilizada para ler o vídeo de saída (sem áudio) e criar um novo vídeo de saída com o áudio do vídeo de entrada original.

O procedimento é bem simples:

Como a função `swap_image_background` já faz todo o processamento de uma imagem, removendo o _background_ verde, acaba sendo necessário apenas ler o vídeo de entrada e processar cada frame (que nada mais são do que imagens) para realizar a remoção do _background_ indesejado e inserção do novo _background_. Após isso o vídeo de saída é salvo e manipulado para inserção do áudio.

### Desenvolvimento do Algoritmo

Primeiro, é necessário importar as bibliotecas que serão utilizadas. A biblioteca `sys` não foi importada, já que não será lido nada da linha de comando neste _notebook_.

In [27]:
import cv2
import os
from image_background_swapper import INPUT_DIR, OUTPUT_DIR, swap_image_background
from moviepy.editor import *

O vídeo de entrada e o novo _background_ para este são obtidos.

In [28]:
INPUT_VIDEO_NAME = 'uau.mp4'
BACKGROUND_IMG_NAME = 'background1.jpg'

input_video = cv2.VideoCapture(INPUT_DIR + INPUT_VIDEO_NAME)
background_img = cv2.imread(INPUT_DIR + BACKGROUND_IMG_NAME)

Algumas informações necessárias são lidas a partir do vídeo de entrada, como as dimensões (largura e altura) e a taxa de quadros.

In [29]:
frame_width = int(input_video.get(3))
frame_height = int(input_video.get(4))
frame_rate = int(input_video.get(cv2.CAP_PROP_FPS))

O vídeo de saída já é criado, pois ele deverá estar aberto durante o processamento do vídeo de entrada, para já ir gravando cada frame (quadro) atualizado.

In [30]:
output_video_path = OUTPUT_DIR + INPUT_VIDEO_NAME

output_video = cv2.VideoWriter(
  output_video_path,
  cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'),
  frame_rate,
  (frame_width,frame_height)
)

OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


O vídeo de entrada é lido frame a frame, sendo que cada frame passa pela operação de troca de _background_ através da função _swap_image_background_. Além disso, em cada iteração do _loop_ o frame atualizado com o novo _background_ é escrito no vídeo de saída.

In [31]:
while(input_video.isOpened()):
  ret, img = input_video.read()

  if not ret:
    break

  result_img = swap_image_background(img, background_img)
  output_video.write(result_img)

Após finalizar as operações de vídeo com o OpenCV, os arquivos são liberados.

In [32]:
input_video.release()
output_video.release()

O vídeo de saída salvo nas operações acima não contém o áudio do vídeo original. O motivo para isso é que o OpenCV é focado no manipulamento de imagens, e não possui funcionalidades para o manipulamento de áudio. Para que o vídeo de saída tenha o áudio original, é necessário inserir esse áudio através de outra biblioteca. É aí que entra o MoviePy, biblioteca que permite manipular o áudio e vídeo de um arquivo.

Portanto, o vídeo de saída precisa ser atualizado, e devido a isso, ele é renomeado para um arquivo temporário, já que o verdadeiro vídeo de saída (com o áudio) será escrito ainda.

In [33]:
temp_output_video_path = OUTPUT_DIR + 'temp_' + INPUT_VIDEO_NAME

os.rename(output_video_path, temp_output_video_path)

O clip de áudio é optido a partir do vídeo de entrada original e atribuído ao vídeo de saída. O vídeo de saída é escrito novamente, porém dessa vez contendo o áudio.

In [34]:
audio_from_input_video = AudioFileClip(INPUT_DIR + INPUT_VIDEO_NAME)

output_video = VideoFileClip(temp_output_video_path)
output_video.audio = audio_from_input_video
output_video.write_videofile(output_video_path)

Moviepy - Building video output/uau.mp4.
MoviePy - Writing audio in uauTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output/uau.mp4



Moviepy - Done !
Moviepy - video ready output/uau.mp4


Por fim, o vídeo de saída temporário é apagado, já que o vídeo de saída com áudio já foi salvo com sucesso.

In [35]:
os.remove(temp_output_video_path)